In [1]:
import numpy as np

In [2]:
import pandas as pd

In [3]:
import matplotlib.pyplot as plt

In [4]:
plt.close('all')

In [5]:
MLBElo = pd.read_csv(r"C:\Users\Eric\Documents\EECS 731\Proj4\data\mlb_elo.csv")

In [6]:
MLBElo

,date,season,neutral,playoff,team1,team2,elo1_pre,elo2_pre,elo_prob1,elo_prob2,...,pitcher1_rgs,pitcher2_rgs,pitcher1_adj,pitcher2_adj,rating_prob1,rating_prob2,rating1_post,rating2_post,score1,score2
0,2020-10-10,2020,1,d,LAD,SDP,1598.290926,1525.832831,0.635558,0.364442,...,NaN,NaN,NaN,NaN,0.635030,0.364970,NaN,NaN,NaN,NaN
1,2020-10-10,2020,1,d,ATL,FLA,1543.508397,1482.006574,0.615867,0.384133,...,NaN,NaN,NaN,NaN,0.629976,0.370024,NaN,NaN,NaN,NaN
2,2020-10-09,2020,1,d,SDP,LAD,1525.832831,1598.290926,0.364442,0.635558,...,NaN,NaN,NaN,NaN,0.364970,0.635030,NaN,NaN,NaN,NaN
3,2020-10-09,2020,1,d,TBD,NYY,1566.075394,1557.931446,0.515622,0.484378,...,NaN,NaN,NaN,NaN,0.507634,0.492366,NaN,NaN,NaN,NaN
4,2020-10-09,2020,1,d,OAK,HOU,1547.618027,1542.711978,0.509413,0.490587,...,NaN,NaN,NaN,NaN,0.482885,0.517115,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220941,1871-05-09,1871,0,NaN,TRO,ATL,1450.000000,1451.483000,0.532359,0.467641,...,NaN,NaN,NaN,NaN,0.522460,0.477540,1448.525,1452.958,5.0,9.0
220942,1871-05-08,1871,0,NaN,CHC,CL1,1450.000000,1451.943000,0.531700,0.468300,...,NaN,NaN,NaN,NaN,0.528402,0.471598,1451.256,1450.687,14.0,12.0
220943,1871-05-06,1871,0,NaN,RC1,CL1,1450.000000,1448.754000,0.536268,0.463732,...,NaN,NaN,NaN,NaN,0.532974,0.467026,1446.810,1451.943,4.0,12.0
220944,1871-05-05,1871,0,NaN,WS3,ATL,1450.000000,1450.000000,0.534484,0.465516,...,NaN,NaN,NaN,NaN,0.534484,0.465516,1448.517,1451.483,18.0,20.0


In [7]:
# where are the values missing
print("*****In the MLBElo set*****")
print(MLBElo.isna().sum())

*****In the MLBElo set*****
date                 0
season               0
neutral              0
playoff         219291
team1                0
team2                0
elo1_pre             0
elo2_pre             0
elo_prob1            0
elo_prob2            0
elo1_post           20
elo2_post           20
rating1_pre          0
rating2_pre          0
pitcher1            56
pitcher2            56
pitcher1_rgs     33759
pitcher2_rgs     33759
pitcher1_adj     34440
pitcher2_adj     34439
rating_prob1         0
rating_prob2         0
rating1_post        20
rating2_post        20
score1              20
score2              20
dtype: int64


In [8]:
# See types of features
MLBElo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220946 entries, 0 to 220945
Data columns (total 26 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   date          220946 non-null  object 
 1   season        220946 non-null  int64  
 2   neutral       220946 non-null  int64  
 3   playoff       1655 non-null    object 
 4   team1         220946 non-null  object 
 5   team2         220946 non-null  object 
 6   elo1_pre      220946 non-null  float64
 7   elo2_pre      220946 non-null  float64
 8   elo_prob1     220946 non-null  float64
 9   elo_prob2     220946 non-null  float64
 10  elo1_post     220926 non-null  float64
 11  elo2_post     220926 non-null  float64
 12  rating1_pre   220946 non-null  float64
 13  rating2_pre   220946 non-null  float64
 14  pitcher1      220890 non-null  object 
 15  pitcher2      220890 non-null  object 
 16  pitcher1_rgs  187187 non-null  float64
 17  pitcher2_rgs  187187 non-null  float64
 18  pitc

In [9]:
# Make new dataframe for 'cleaned' data
cleanMLB = MLBElo

In [10]:
#Remove columns per report
del cleanMLB['date']
del cleanMLB['season']
del cleanMLB['pitcher1']
del cleanMLB['pitcher2']
del cleanMLB['elo_prob1']
del cleanMLB['elo_prob2']


In [11]:
cleanMLB.head()

,neutral,playoff,team1,team2,elo1_pre,elo2_pre,elo1_post,elo2_post,rating1_pre,rating2_pre,pitcher1_rgs,pitcher2_rgs,pitcher1_adj,pitcher2_adj,rating_prob1,rating_prob2,rating1_post,rating2_post,score1,score2
0,1,d,LAD,SDP,1598.290926,1525.832831,NaN,NaN,1606.065971,1534.038131,NaN,NaN,NaN,NaN,0.635030,0.364970,NaN,NaN,NaN,NaN
1,1,d,ATL,FLA,1543.508397,1482.006574,NaN,NaN,1541.398290,1475.108982,NaN,NaN,NaN,NaN,0.629976,0.370024,NaN,NaN,NaN,NaN
2,1,d,SDP,LAD,1525.832831,1598.290926,NaN,NaN,1534.038131,1606.065971,NaN,NaN,NaN,NaN,0.364970,0.635030,NaN,NaN,NaN,NaN
3,1,d,TBD,NYY,1566.075394,1557.931446,NaN,NaN,1562.032965,1562.805994,NaN,NaN,NaN,NaN,0.507634,0.492366,NaN,NaN,NaN,NaN
4,1,d,OAK,HOU,1547.618027,1542.711978,NaN,NaN,1532.981409,1552.953477,NaN,NaN,NaN,NaN,0.482885,0.517115,NaN,NaN,NaN,NaN


In [12]:
#Testing one-hot encoding for team name
test = pd.get_dummies(cleanMLB)
test

,neutral,elo1_pre,elo2_pre,elo1_post,elo2_post,rating1_pre,rating2_pre,pitcher1_rgs,pitcher2_rgs,pitcher1_adj,...,team2_TRO,team2_WOR,team2_WS3,team2_WS4,team2_WS5,team2_WS6,team2_WS7,team2_WS8,team2_WS9,team2_WSN
0,1,1598.290926,1525.832831,NaN,NaN,1606.065971,1534.038131,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1,1,1543.508397,1482.006574,NaN,NaN,1541.398290,1475.108982,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,1,1525.832831,1598.290926,NaN,NaN,1534.038131,1606.065971,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,1,1566.075394,1557.931446,NaN,NaN,1562.032965,1562.805994,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,1,1547.618027,1542.711978,NaN,NaN,1532.981409,1552.953477,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220941,0,1450.000000,1451.483000,1448.525,1452.958,1450.000000,1451.483000,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
220942,0,1450.000000,1451.943000,1451.256,1450.687,1450.000000,1451.943000,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
220943,0,1450.000000,1448.754000,1446.810,1451.943,1450.000000,1448.754000,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
220944,0,1450.000000,1450.000000,1448.517,1451.483,1450.000000,1450.000000,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [14]:
cleanMLB = test
cleanMLB.head()

,neutral,elo1_pre,elo2_pre,elo1_post,elo2_post,rating1_pre,rating2_pre,pitcher1_rgs,pitcher2_rgs,pitcher1_adj,...,team2_TRO,team2_WOR,team2_WS3,team2_WS4,team2_WS5,team2_WS6,team2_WS7,team2_WS8,team2_WS9,team2_WSN
0,1,1598.290926,1525.832831,NaN,NaN,1606.065971,1534.038131,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1,1,1543.508397,1482.006574,NaN,NaN,1541.398290,1475.108982,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,1,1525.832831,1598.290926,NaN,NaN,1534.038131,1606.065971,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,1,1566.075394,1557.931446,NaN,NaN,1562.032965,1562.805994,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,1,1547.618027,1542.711978,NaN,NaN,1532.981409,1552.953477,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
